In [2]:
import os  # for file paths
import pandas as pd
import awswrangler as wr
import pydbtools as pydb  # see https://github.com/moj-analytical-services/pydbtools
import altair as alt  # for plotting, see https://altair-viz.github.io/getting_started/overview.html

# few things for viewing dataframes better
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 900)
pd.set_option("display.max_colwidth", 200)

In [3]:
# define some variables to be used throughout the notebook
database = "familyman_dev_v2"
snapshot_date = "2021-07-08"
database2 = "fcsq"

In [35]:
#Applications by type - children involved
ac = f"""
SELECT
  Year,
  CAST(Quarter AS VARCHAR) Quarter,
  'Application' AS Type,
  'Children' AS Count_type,
  CASE WHEN Case_type = 'C' THEN 'Public law'
       WHEN Case_type = 'P' THEN 'Private law'
    End AS Public_private,
  'N/A' AS Disposal_type,
   Order_desc AS Order_type,
   CAST(Order_code AS VARCHAR) Order_type_code,
  'N/A' AS Gender,
  'N/A' AS age_band,
  'N/A' AS Applicants_in_case,
  'N/A' AS Respondents_in_case,
  'N/A' AS HC_INDICATOR,
   COUNT(*) AS Count
FROM
  {database2}.ca_apps_child
Where
  Year > 2010
Group BY
  Year,
  Quarter,
  Case_type,
  Order_desc,
  Order_code
"""

pydb.create_temp_table(ac, "ca_apps_type_child")

In [36]:
#Orders applied for (by order type)
ao = f"""
SELECT
  Year,
  CAST(Quarter AS VARCHAR) Quarter,
  'Application' AS Type,
  'Order type' AS Count_type,
  CASE WHEN Case_type = 'C' THEN 'Public law'
       WHEN Case_type = 'P' THEN 'Private law'
    End AS Public_private,
  'N/A' AS Disposal_type,
   Order_desc,
   CAST(Order_code AS VARCHAR) Order_type_code,
  'N/A' AS Gender,
  'N/A' AS age_band,
  'N/A' AS Applicants_in_case,
  'N/A' AS Respondents_in_case,
  'N/A' AS HC_INDICATOR,
   COUNT(*) AS Count
FROM
  __temp__.ca_apps_order_count
Where
  Year > 2010
Group BY
  Year,
  Quarter,
  Case_type,
  Order_desc,
  Order_code
"""

pydb.create_temp_table(ao, "ca_apps_type_order")

In [37]:
#Application events
ae = f"""
SELECT
  Year,
  CAST(Quarter AS VARCHAR) Quarter,
  'Application' AS Type,
  'Application events' AS Count_type,
  CASE WHEN Case_type = 'C' THEN 'Public law'
       WHEN Case_type = 'P' THEN 'Private law'
    End AS Public_private,
  'N/A' AS Disposal_type,
   'All' AS Order_type,
   ' ' AS Order_type_code,
  'N/A' AS Gender,
  'N/A' AS age_band,
  'N/A' AS Applicants_in_case,
  'N/A' AS Respondents_in_case,
  'N/A' AS HC_INDICATOR,
   COUNT(*) AS Count
FROM
  __temp__.ca_apps_event_count
Where
  Year > 2010
Group BY
  Year,
  Quarter,
  Case_type
"""

pydb.create_temp_table(ae, "ca_apps_events")

In [38]:
#Case starts - including number of applicants/respondents and Hight court indicator*/
ap = f"""
SELECT
  P.Year,
  CAST(P.Quarter AS VARCHAR) Quarter,
  'Cases' AS Type,
  'Cases starting' AS Count_type,
  CASE WHEN P.Case_type = 'C' THEN 'Public law'
       WHEN P.Case_type = 'P' THEN 'Private law'
    End AS Public_private,
  'N/A' AS Disposal_type,
  'All' AS Order_type,
   ' ' AS Order_type_code,
  'N/A' AS Gender,
  'N/A' AS age_band,
   Case When P.no_of_applicants = 1 THEN '1'
      ELSE '2+'
    END AS Applicants_in_case,
   Case When P.no_of_respondents in (1,2,3)
         THEN CAST(P.no_of_respondents AS VARCHAR)
         ELSE '4+'
        END AS Respondents_in_case,
   CASE WHEN HC.case_hc_value IS NULL THEN 'No'
          ELSE HC.case_hc_value
     END AS HC_INDICATOR,
   COUNT(*) AS Count
FROM
  __temp__.party_count P
  left join __temp__.ca_cases_HC HC
    on P.Case_number = HC.Case_number
     and P.case_type = HC.case_type
Where
  P.Year > 2010
Group BY
  P.Year,
  P.Quarter,
  p.case_type,
  Case When P.no_of_applicants = 1 THEN '1'
      ELSE '2+'
    END,
  Case When P.no_of_respondents in (1,2,3)
         THEN CAST(P.no_of_respondents AS VARCHAR)
         ELSE '4+'
        END,      
  CASE WHEN HC.case_hc_value IS NULL THEN 'No'
          ELSE HC.case_hc_value
     END
"""

pydb.create_temp_table(ap, "ca_apps_cases")

In [42]:
#Individual children - annual - including age of child
ci = f"""
SELECT
  Year,
  ' ' AS Quarter,
  'Application' AS Type,
  'Individual children' AS Count_type,
  CASE WHEN Case_type = 'C' THEN 'Public law'
       WHEN Case_type = 'P' THEN 'Private law'
    End AS Public_private,
  'N/A' AS Disposal_type,
  'All' AS Order_type,
  ' ' AS Order_type_code,
  Gender,
  age_band,
  'N/A' AS Applicants_in_case,
  'N/A' AS Respondents_in_case,
  'N/A' AS HC_INDICATOR,
  COUNT(*) AS Count
FROM
  __temp__.ca_apps_ind_child_yr
Where
  Year > 2010
Group BY
  Year,
  Case_type,
  Gender,
  age_band
"""

pydb.create_temp_table(ci, "ca_apps_ind_yr")

In [43]:
#Individual children - quarterly - including age of child
cq = f"""
SELECT
  Year,
  CAST(Quarter AS VARCHAR) Quarter,
  'Application' AS Type,
  'Individual children' AS Count_type,
  CASE WHEN Case_type = 'C' THEN 'Public law'
       WHEN Case_type = 'P' THEN 'Private law'
    End AS Public_private,
  'N/A' AS Disposal_type,
  'All' AS Order_type,
  ' ' AS Order_type_code,
  Gender,
  age_band,
  'N/A' AS Applicants_in_case,
  'N/A' AS Respondents_in_case,
  'N/A' AS HC_INDICATOR,
  COUNT(*) AS Count
FROM
  __temp__.ca_apps_ind_child_qtr
Where
  Year > 2010
Group BY
  Year,
  Quarter,
  Case_type,
  Gender,
  age_band
"""

pydb.create_temp_table(cq, "ca_apps_ind_qtr")

In [44]:
#Union together datasets
ca_csv = f"""
SELECT
  *
FROM
  __temp__.CA_APPS_TYPE_CHILD
UNION ALL
SELECT
  *
FROM
  __temp__.CA_APPS_TYPE_ORDER
UNION ALL
SELECT
  *
FROM
  __temp__.CA_APPS_EVENTS
UNION ALL
SELECT
  *
FROM
  __temp__.CA_APPS_CASES
UNION ALL
SELECT
  *
FROM
  __temp__.CA_APPS_IND_YR
UNION ALL
SELECT
  *
FROM
  __temp__.CA_APPS_IND_QTR

"""
pydb.create_temp_table(ca_csv, "ca_apps_csv")

In [46]:
pydb.read_sql_query ("select * from __temp__.ca_apps_csv")

,year,quarter,type,count_type,public_private,disposal_type,order_type,order_type_code,gender,age_band,applicants_in_case,respondents_in_case,hc_indicator,count
0,2015,1,Cases,Cases starting,Public law,N/A,All,,N/A,N/A,1,3,No,413
1,2014,1,Cases,Cases starting,Private law,N/A,All,,N/A,N/A,2+,2,No,368
2,2020,2,Cases,Cases starting,Private law,N/A,All,,N/A,N/A,2+,2,No,192
3,2011,2,Cases,Cases starting,Private law,N/A,All,,N/A,N/A,2+,2,No,262
4,2019,2,Cases,Cases starting,Private law,N/A,All,,N/A,N/A,2+,4+,No,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6515,2019,1,Application,Children,Private law,N/A,Parental Order,28,N/A,N/A,N/A,N/A,N/A,149
6516,2014,1,Application,Children,Public law,N/A,Extension of Emergency Protection Order,15,N/A,N/A,N/A,N/A,N/A,19
6517,2015,3,Application,Children,Private law,N/A,Enforcement - Reduced Hours,35,N/A,N/A,N/A,N/A,N/A,12
6518,2018,2,Application,Children,Private law,N/A,Enforcement - Change of Area,35,N/A,N/A,N/A,N/A,N/A,14


In [52]:
data = pydb.read_sql_query ("select * from __temp__.ca_apps_csv")

In [58]:
df = pd.DataFrame(data)

In [60]:
df.to_csv (r'S:\hq\102PF\Shared\Group_LCDSHD2_IMD\IMD\Statistics Branch\Family\FamilyMan for Analytical Platform\Moving to the platform\CA Applications\ca_apps.csv', header = True)

,year,quarter,type,count_type,public_private,disposal_type,order_type,order_type_code,gender,age_band,applicants_in_case,respondents_in_case,hc_indicator,count
0,2020,2,Application,Individual children,Public law,N/A,All,,Female,Other,N/A,N/A,N/A,5
1,2011,3,Application,Individual children,Public law,N/A,All,,Female,Other,N/A,N/A,N/A,7
2,2011,1,Application,Individual children,Public law,N/A,All,,Unknown,Unknown,N/A,N/A,N/A,1
3,2019,4,Application,Individual children,Private law,N/A,All,,Female,10-14 years,N/A,N/A,N/A,2312
4,2012,3,Application,Individual children,Private law,N/A,All,,Male,<1 year,N/A,N/A,N/A,687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6515,2019,4,Cases,Cases starting,Public law,N/A,All,,N/A,N/A,1,1,Y,29
6516,2019,2,Cases,Cases starting,Private law,N/A,All,,N/A,N/A,2+,1,No,248
6517,2019,4,Cases,Cases starting,Public law,N/A,All,,N/A,N/A,2+,1,No,19
6518,2019,2,Cases,Cases starting,Public law,N/A,All,,N/A,N/A,1,1,Y,28
